In [44]:
import os
import json
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

from src.data import data_layer
from src.model.model_utils import calculateStatistics
from src.model.model_GAN import FeedForwardModelWithNA_GAN_Ensembled, FeedForwardModelWithNA_GAN
from src.model.model_RtnFcst import FeedForwardModelWithNA_Return_Ensembled
from src.utils import deco_print
from src.utils import load_dataframe
from src.utils import sharpe

import tensorflow.contrib.slim as slim

In [10]:
def model_summary():
    model_vars = tf.trainable_variables()
    slim.model_analyzer.analyze_vars(model_vars, print_info=True)

##### Model Configurations

In [3]:
task_id = 1
logdirs = ['output/Task-%d-Trial-%d/sharpe' %(task_id, k) for k in range(9)]
with open('config/config.json') as file:
    config = json.load(file)
    
logdirs_RF = ['output_RF/Task-%d-Trial-%d' %(task_id, k) for k in range(9)]
with open('config_RF/config_RF_%d.json'%(task_id)) as file:
    config_RF = json.load(file)

In [7]:
!ls output/sharpe

GAN  checkpoint			     model-best.index
UNC  model-best.data-00000-of-00001  model-best.meta


In [4]:
logdirs

['output/Task-1-Trial-0/sharpe',
 'output/Task-1-Trial-1/sharpe',
 'output/Task-1-Trial-2/sharpe',
 'output/Task-1-Trial-3/sharpe',
 'output/Task-1-Trial-4/sharpe',
 'output/Task-1-Trial-5/sharpe',
 'output/Task-1-Trial-6/sharpe',
 'output/Task-1-Trial-7/sharpe',
 'output/Task-1-Trial-8/sharpe']

In [22]:
logdirs = ["output/sharpe"]

##### Load Data

In [8]:
dl = data_layer.DataInRamInputLayer(config['individual_feature_file'],
                                    pathMacroFeature=config['macro_feature_file'])
meanMacroFeature, stdMacroFeature = dl.getMacroFeatureMeanStd()
dl_valid = data_layer.DataInRamInputLayer(config['individual_feature_file_valid'],
                                          pathMacroFeature=config['macro_feature_file_valid'],
                                          meanMacroFeature=meanMacroFeature,
                                          stdMacroFeature=stdMacroFeature)
dl_test = data_layer.DataInRamInputLayer(config['individual_feature_file_test'],
                                         pathMacroFeature=config['macro_feature_file_test'],
                                         meanMacroFeature=meanMacroFeature,
                                         stdMacroFeature=stdMacroFeature)

In [42]:
dl.getIndividualFeatureByIdx(0)

'A2ME'

In [39]:
help(dl)

Help on DataInRamInputLayer in module src.data.data_layer object:

class DataInRamInputLayer(builtins.object)
 |  Methods defined here:
 |  
 |  __init__(self, pathIndividualFeature, pathMacroFeature=None, macroIdx=None, meanMacroFeature=None, stdMacroFeature=None, normalizeMacroFeature=True)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  getDateByIdx(self, idx)
 |  
 |  getDateCountList(self)
 |  
 |  getDateList(self)
 |  
 |  getFeatureByIdx(self, idx)
 |  
 |  getIdxByIndividualFeature(self, var)
 |  
 |  getIndividualFeatureByIdx(self, idx)
 |  
 |  getIndividualFeatureColarLabelMap(self)
 |  
 |  getIndividualFeatureList(self)
 |  
 |  getMacroFeatureByIdx(self, idx)
 |  
 |  getMacroFeatureList(self)
 |  
 |  getMacroFeatureMeanStd(self)
 |  
 |  iterateOneEpoch(self, subEpoch=False)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance 

In [47]:
# sample model
sample_model = FeedForwardModelWithNA_GAN(config, 'test', config['tSize'], 
                                          force_var_reuse=True, global_step=global_step)

In [48]:
model_summary()

---------
Variables: name (type shape) [size]
---------
Model_Layer/RNN_Layer/rnn/lstm_cell/kernel:0 (float32_ref 182x16) [2912, bytes: 11648]
Model_Layer/RNN_Layer/rnn/lstm_cell/bias:0 (float32_ref 16) [16, bytes: 64]
Model_Layer/NN_Layer/dense_layer_0/dense/kernel:0 (float32_ref 50x64) [3200, bytes: 12800]
Model_Layer/NN_Layer/dense_layer_0/dense/bias:0 (float32_ref 64) [64, bytes: 256]
Model_Layer/NN_Layer/dense_layer_1/dense/kernel:0 (float32_ref 64x64) [4096, bytes: 16384]
Model_Layer/NN_Layer/dense_layer_1/dense/bias:0 (float32_ref 64) [64, bytes: 256]
Model_Layer/NN_Layer/last_dense_layer/dense/kernel:0 (float32_ref 64x1) [64, bytes: 256]
Model_Layer/NN_Layer/last_dense_layer/dense/bias:0 (float32_ref 1) [1, bytes: 4]
Moment_Layer/RNN_Layer/rnn/lstm_cell/kernel:0 (float32_ref 210x128) [26880, bytes: 107520]
Moment_Layer/RNN_Layer/rnn/lstm_cell/bias:0 (float32_ref 128) [128, bytes: 512]
Moment_Layer/NN_Layer/last_dense_layer/dense/kernel:0 (float32_ref 78x8) [624, bytes: 2496]
Mo

##### Load Model

In [23]:
tf.reset_default_graph()
global_step = tf.train.get_or_create_global_step()
model = FeedForwardModelWithNA_GAN_Ensembled(logdirs, config, 'test', config['tSize'], global_step=global_step)

In [24]:
model_summary()

---------
Variables: name (type shape) [size]
---------
Model_Layer/RNN_Layer/rnn/lstm_cell/kernel:0 (float32_ref 182x16) [2912, bytes: 11648]
Model_Layer/RNN_Layer/rnn/lstm_cell/bias:0 (float32_ref 16) [16, bytes: 64]
Model_Layer/NN_Layer/dense_layer_0/dense/kernel:0 (float32_ref 50x64) [3200, bytes: 12800]
Model_Layer/NN_Layer/dense_layer_0/dense/bias:0 (float32_ref 64) [64, bytes: 256]
Model_Layer/NN_Layer/dense_layer_1/dense/kernel:0 (float32_ref 64x64) [4096, bytes: 16384]
Model_Layer/NN_Layer/dense_layer_1/dense/bias:0 (float32_ref 64) [64, bytes: 256]
Model_Layer/NN_Layer/last_dense_layer/dense/kernel:0 (float32_ref 64x1) [64, bytes: 256]
Model_Layer/NN_Layer/last_dense_layer/dense/bias:0 (float32_ref 1) [1, bytes: 4]
Moment_Layer/RNN_Layer/rnn/lstm_cell/kernel:0 (float32_ref 210x128) [26880, bytes: 107520]
Moment_Layer/RNN_Layer/rnn/lstm_cell/bias:0 (float32_ref 128) [128, bytes: 512]
Moment_Layer/NN_Layer/last_dense_layer/dense/kernel:0 (float32_ref 78x8) [624, bytes: 2496]
Mo

In [25]:
model_valid = FeedForwardModelWithNA_GAN_Ensembled(logdirs, config, 'test', config['tSize_valid'], force_var_reuse=True, global_step=global_step)

In [26]:
model_summary()

---------
Variables: name (type shape) [size]
---------
Model_Layer/RNN_Layer/rnn/lstm_cell/kernel:0 (float32_ref 182x16) [2912, bytes: 11648]
Model_Layer/RNN_Layer/rnn/lstm_cell/bias:0 (float32_ref 16) [16, bytes: 64]
Model_Layer/NN_Layer/dense_layer_0/dense/kernel:0 (float32_ref 50x64) [3200, bytes: 12800]
Model_Layer/NN_Layer/dense_layer_0/dense/bias:0 (float32_ref 64) [64, bytes: 256]
Model_Layer/NN_Layer/dense_layer_1/dense/kernel:0 (float32_ref 64x64) [4096, bytes: 16384]
Model_Layer/NN_Layer/dense_layer_1/dense/bias:0 (float32_ref 64) [64, bytes: 256]
Model_Layer/NN_Layer/last_dense_layer/dense/kernel:0 (float32_ref 64x1) [64, bytes: 256]
Model_Layer/NN_Layer/last_dense_layer/dense/bias:0 (float32_ref 1) [1, bytes: 4]
Moment_Layer/RNN_Layer/rnn/lstm_cell/kernel:0 (float32_ref 210x128) [26880, bytes: 107520]
Moment_Layer/RNN_Layer/rnn/lstm_cell/bias:0 (float32_ref 128) [128, bytes: 512]
Moment_Layer/NN_Layer/last_dense_layer/dense/kernel:0 (float32_ref 78x8) [624, bytes: 2496]
Mo

In [27]:
model_test = FeedForwardModelWithNA_GAN_Ensembled(logdirs, config, 'test', config['tSize_test'], force_var_reuse=True, global_step=global_step)

In [28]:
model_summary()

---------
Variables: name (type shape) [size]
---------
Model_Layer/RNN_Layer/rnn/lstm_cell/kernel:0 (float32_ref 182x16) [2912, bytes: 11648]
Model_Layer/RNN_Layer/rnn/lstm_cell/bias:0 (float32_ref 16) [16, bytes: 64]
Model_Layer/NN_Layer/dense_layer_0/dense/kernel:0 (float32_ref 50x64) [3200, bytes: 12800]
Model_Layer/NN_Layer/dense_layer_0/dense/bias:0 (float32_ref 64) [64, bytes: 256]
Model_Layer/NN_Layer/dense_layer_1/dense/kernel:0 (float32_ref 64x64) [4096, bytes: 16384]
Model_Layer/NN_Layer/dense_layer_1/dense/bias:0 (float32_ref 64) [64, bytes: 256]
Model_Layer/NN_Layer/last_dense_layer/dense/kernel:0 (float32_ref 64x1) [64, bytes: 256]
Model_Layer/NN_Layer/last_dense_layer/dense/bias:0 (float32_ref 1) [1, bytes: 4]
Moment_Layer/RNN_Layer/rnn/lstm_cell/kernel:0 (float32_ref 210x128) [26880, bytes: 107520]
Moment_Layer/RNN_Layer/rnn/lstm_cell/bias:0 (float32_ref 128) [128, bytes: 512]
Moment_Layer/NN_Layer/last_dense_layer/dense/kernel:0 (float32_ref 78x8) [624, bytes: 2496]
Mo

In [29]:
model_plot = FeedForwardModelWithNA_GAN_Ensembled(logdirs, config, 'plot', 1, force_var_reuse=True, global_step=global_step)

In [30]:
model_summary()

---------
Variables: name (type shape) [size]
---------
Model_Layer/RNN_Layer/rnn/lstm_cell/kernel:0 (float32_ref 182x16) [2912, bytes: 11648]
Model_Layer/RNN_Layer/rnn/lstm_cell/bias:0 (float32_ref 16) [16, bytes: 64]
Model_Layer/NN_Layer/dense_layer_0/dense/kernel:0 (float32_ref 50x64) [3200, bytes: 12800]
Model_Layer/NN_Layer/dense_layer_0/dense/bias:0 (float32_ref 64) [64, bytes: 256]
Model_Layer/NN_Layer/dense_layer_1/dense/kernel:0 (float32_ref 64x64) [4096, bytes: 16384]
Model_Layer/NN_Layer/dense_layer_1/dense/bias:0 (float32_ref 64) [64, bytes: 256]
Model_Layer/NN_Layer/last_dense_layer/dense/kernel:0 (float32_ref 64x1) [64, bytes: 256]
Model_Layer/NN_Layer/last_dense_layer/dense/bias:0 (float32_ref 1) [1, bytes: 4]
Moment_Layer/RNN_Layer/rnn/lstm_cell/kernel:0 (float32_ref 210x128) [26880, bytes: 107520]
Moment_Layer/RNN_Layer/rnn/lstm_cell/bias:0 (float32_ref 128) [128, bytes: 512]
Moment_Layer/NN_Layer/last_dense_layer/dense/kernel:0 (float32_ref 78x8) [624, bytes: 2496]
Mo

In [31]:
gpu_options = tf.GPUOptions(allow_growth=True)
sess_config = tf.ConfigProto(gpu_options=gpu_options)
sess = tf.Session(config=sess_config)

In [32]:
INITIAL_train = model.getZeroInitialState()
INITIAL_valid = model.getNextInitialState(sess, dl, initial_state=INITIAL_train)
INITIAL_test = model_valid.getNextInitialState(sess, dl_valid, initial_state=INITIAL_valid)

INFO:tensorflow:Restoring parameters from output/sharpe/model-best
>==================> Restored checkpoint
INFO:tensorflow:Restoring parameters from output/sharpe/model-best
>==================> Restored checkpoint


##### Model Performance

In [33]:
w = model.getWeightWithData(sess, dl, initial_state=INITIAL_train, normalized=True)
w_valid = model_valid.getWeightWithData(sess, dl_valid, initial_state=INITIAL_valid, normalized=True)
w_test = model_test.getWeightWithData(sess, dl_test, initial_state=INITIAL_test, normalized=True)

INFO:tensorflow:Restoring parameters from output/sharpe/model-best
>==================> Restored checkpoint
INFO:tensorflow:Restoring parameters from output/sharpe/model-best
>==================> Restored checkpoint
INFO:tensorflow:Restoring parameters from output/sharpe/model-best
>==================> Restored checkpoint


In [36]:
Ftrain = model.getNormalizedSDFFactor(sess, dl, initial_state=INITIAL_train)
Fvalid = model_valid.getNormalizedSDFFactor(sess, dl_valid, initial_state=INITIAL_valid)
Ftest = model_test.getNormalizedSDFFactor(sess, dl_test, initial_state=INITIAL_test)
sdf_norm_ensemble = np.concatenate([Ftrain, Fvalid, Ftest])
np.save(os.path.join(logdirs[0], 'sdf_normalized_ensemble.npy'), sdf_norm_ensemble)

INFO:tensorflow:Restoring parameters from output/sharpe/model-best
>==================> Restored checkpoint
INFO:tensorflow:Restoring parameters from output/sharpe/model-best
>==================> Restored checkpoint
INFO:tensorflow:Restoring parameters from output/sharpe/model-best
>==================> Restored checkpoint


In [37]:
SR_train = sharpe(Ftrain)
SR_valid = sharpe(Fvalid)
SR_test = sharpe(Ftest)
deco_print('SDF Portfolio Sharpe Ratio: Train %0.2f\tValid %0.2f\tTest %0.2f' %(SR_train, SR_valid, SR_test))

>==================> SDF Portfolio Sharpe Ratio: Train 2.48	Valid 0.80	Test 0.42


In [8]:
SR_train = sharpe(Ftrain)
SR_valid = sharpe(Fvalid)
SR_test = sharpe(Ftest)
deco_print('SDF Portfolio Sharpe Ratio: Train %0.2f\tValid %0.2f\tTest %0.2f' %(SR_train, SR_valid, SR_test))

>==================> SDF Portfolio Sharpe Ratio: Train 3.02	Valid 1.39	Test 0.77


##### Predictive Performance

In [9]:
tf.reset_default_graph()
global_step = tf.train.get_or_create_global_step()
model_plot_RF = FeedForwardModelWithNA_Return_Ensembled(logdirs_RF, config_RF, 'plot', global_step=global_step)
gpu_options = tf.GPUOptions(allow_growth=True)
sess_config = tf.ConfigProto(gpu_options=gpu_options)
sess = tf.Session(config=sess_config)

In [10]:
dl_RF_train = data_layer.DataInRamInputLayer(config['individual_feature_file'])
dl_RF_valid = data_layer.DataInRamInputLayer(config['individual_feature_file_valid'])
dl_RF_test = data_layer.DataInRamInputLayer(config['individual_feature_file_test'])

In [ ]:
beta_train = model_plot_RF.getPrediction(sess, dl_RF_train)
beta_valid = model_plot_RF.getPrediction(sess, dl_RF_valid)
beta_test = model_plot_RF.getPrediction(sess, dl_RF_test)

In [12]:
# EV, XS-R2
EV_train, XSR2_train, WXSR2_train = calculateStatistics(beta_train, dl)
EV_valid, XSR2_valid, WXSR2_valid = calculateStatistics(beta_valid, dl_valid)
EV_test, XSR2_test, WXSR2_test = calculateStatistics(beta_test, dl_test)

In [13]:
deco_print('Explained Variation: Train %0.2f\tValid %0.2f\tTest %0.2f' %(EV_train, EV_valid, EV_test))
deco_print('XS-R2: Train %0.2f\tValid %0.2f\tTest %0.2f' %(XSR2_train, XSR2_valid, XSR2_test))
deco_print('(Weighted) XS-R2: Train %0.2f\tValid %0.2f\tTest %0.2f' %(WXSR2_train, WXSR2_valid, WXSR2_test))

>==================> Explained Variation: Train 0.18	Valid 0.08	Test 0.07
>==================> XS-R2: Train -0.02	Valid 0.04	Test 0.04
>==================> (Weighted) XS-R2: Train 0.12	Valid 0.00	Test 0.22
